In [1]:
import os; os.chdir('/Users/sqian/MKTSRV')
import pandas as pd; pd.set_option('mode.chained_assignment',None)
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import datetime
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy
import re
import xlrd
from datetime import date
from xlsxwriter.utility import xl_rowcol_to_cell
from random import sample
from math import log10, floor
from src.data_models.smartshelper import metrics, fixNum, unSMART, cparty, littleQA, totalUP
from src.data_models.SmartsDataModel import SmartsDataModel
from src.utility.DataModelUtility import execute_query_data_frame

/Users/sqian/.pyenv/versions/3.7.0/envs/env_370/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### 1) Import & Create Datetime & match columns

In [2]:
prod = pd.read_excel('~/Documents/vmfldr/Alerts-AllRows_gemini_20190823-20190828_parity_prod.xlsx')
uat = pd.read_excel('~/Documents/vmfldr/Alerts-AllRows_gemini_20190823-20190828_parity_test.xlsx')

In [39]:
prod['Datetime'] = pd.to_datetime(prod['Date'].astype(str) + ' ' + prod['Time'].astype(str)) 
uat['Datetime'] = pd.to_datetime(uat['Date'].astype(str) + ' ' + uat['Time'].astype(str)) 

prod['dtmatch'] = prod['Datetime'].astype(str) + '__' + prod['Alert Code'].astype(str) + '_' + prod['Account ID Code'].fillna(0).astype(int).astype(str) + '_' + prod['Instrument Code'].fillna('_')
uat['dtmatch'] = uat['Datetime'].astype(str) + '__' + uat['Alert Code'].astype(str) + '_' + uat['Account ID Code'].fillna(0).astype(int).astype(str) + '_' + uat['Instrument Code'].fillna('_')

#### 2) Filter to this year

In [29]:
dateoi = '2019-01-01'
dayprod = prod.loc[prod['Date'] >= dateoi]
dayuat = uat.loc[uat['Date'] >= dateoi]

#### 3) Alert Generation Reconcilliation

In [32]:
dayprodtest = dayprod.groupby(['Date','Time','Alert Code','Account ID Code', 'Instrument Code']).count()['Alert ID']
dayuattest = dayuat.groupby(['Date','Time','Alert Code','Account ID Code', 'Instrument Code']).count()['Alert ID']

testdiff = pd.concat([pd.DataFrame(dayprodtest),pd.DataFrame(dayuattest)], axis=1,join='outer',sort=True).fillna(0).astype(int)
testdiff.columns = ['dayprodtest','dayuattest']
testdiff['count_diff'] = testdiff['dayprodtest'] - testdiff['dayuattest']

discrep = testdiff.loc[testdiff['count_diff'] != 0]
crep = testdiff.loc[testdiff['count_diff'] == 0]

#### 4) Alert Generation Matches - Content Reconcilliation

In [33]:
tempcrep = crep.reset_index()
tempcrep['Datetime'] = pd.to_datetime(tempcrep['Date'].astype(str) + ' ' + tempcrep['Time'].astype(str))
tempcrep['dtmatch'] = tempcrep['Datetime'].astype(str) + '__' + tempcrep['Alert Code'].astype(str)+ '_' + tempcrep['Account ID Code'].fillna(0).astype(int).astype(str)  + '_' + tempcrep['Instrument Code'].fillna('_')

In [35]:
fieldoi = 'Long Text'
collist = ['Datetime','Instrument Code','Account ID Code','Long Text','Reissue Count','uatLong Text']

In [36]:
crepprod = dayprod.loc[(dayprod['dtmatch'].isin(tempcrep['dtmatch']))]
crepuat = dayuat.loc[(dayuat['dtmatch'].isin(tempcrep['dtmatch']))]
# crepprod

In [37]:
mapdict = crepuat[['dtmatch','Long Text']].set_index('dtmatch').to_dict()['Long Text']
crepprod['uatLong Text'] = crepprod['dtmatch'].map(mapdict)
discrep2 = crepprod.loc[crepprod['uatLong Text'] != crepprod['Long Text']][collist]

In [38]:
os.chdir('/Users/sqian/Desktop')

writer = pd.ExcelWriter('UATvsPROD.xlsx', engine='xlsxwriter')

if (len(discrep) == 0): print('empty: discrep')
else: discrep.to_excel(writer, 'AlertGeneration_discrp')

if (len(crep) == 0): print('empty: crep')
else: crep.to_excel(writer, 'AlertGeneration_matches')

if (len(discrep2) == 0): print('empty: discrep2')
else: discrep2.to_excel(writer, 'AlertContent_discrep')

writer.save()

empty: discrep
empty: discrep2
